In [1]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os
import warnings
import time
import matplotlib.pyplot as plt
from importlib import reload

reload(Model)
warnings.simplefilter('ignore', FutureWarning)

# positive numbers, start should be lower than end
# a rebalance index represents the month before the rebalancing takes place
# so returns are calculated starting at rebalanceIndex + 1
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    indexes.reverse()
    return (indexes)


# get ln returns for an equally balanced portfolio of stocks
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	if len(portfolio) - nancount == 0:
		total = 0
	else:
		total = returns/(len(portfolio)-nancount)
	print("Start Date: " + str(Model.convertIndexToDate(-1 * index + 1)))
	print("Total Return: " +str(total))
	return (total)


# make a portfolio with predicted probabilities higher than a hardcoded threshold
def makePortfolio(treeTuple):
	featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Financials")
	probabilityThreshold = 0.85
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	if len(stockTuples) == 0:
		print("No portfolio, probabilities lower than threshold of " + str(probabilityThreshold))
		return 0
	stocks, probabilities = zip(*stockTuples)
	return(getReturns(stocks, treeTuple[0], 3))

In [2]:
def poolBuild(trainIndexes):
    t = time.time()
    featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
    forest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = trainIndexes, target= 'Rate of Return', features = featureList, \
                                               featureLength = 12, targetLength = 3, sector = "Financials", percentileTarget = 90, percentileAvoid = 10)
    print("Build time: " + str(time.time()-t) + " seconds.")
    return forest

In [3]:
indexes = []
for i in rebalanceIndexes(4,28):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-340, -337, -334, -331, -328, -325, -322, -319, -316, -313, -310,
       -307, -304, -301, -298, -295, -292, -289, -286, -283, -280, -277,
       -274, -271, -268, -265, -262, -259, -256, -253, -250, -247, -244,
       -241, -238, -235, -232, -229, -226, -223, -220, -217, -214, -211,
       -208, -205, -202, -199, -196, -193, -190, -187, -184, -181, -178,
       -175, -172, -169, -166, -163, -160, -157, -154, -151, -148, -145,
       -142, -139, -136, -133, -130, -127, -124, -121, -118, -115, -112,
       -109, -106, -103, -100,  -97,  -94,  -91,  -88,  -85,  -82,  -79,
        -76,  -73,  -70,  -67,  -64,  -61,  -58,  -55,  -52,  -49,  -46,
        -43]), array([-337, -334, -331, -328, -325, -322, -319, -316, -313, -310, -307,
       -304, -301, -298, -295, -292, -289, -286, -283, -280, -277, -274,
       -271, -268, -265, -262, -259, -256, -253, -250, -247, -244, -241,
       -238, -235, -232, -229, -226, -223, -220, -217, -214, -211, -208,
       -205, -202, -199, -196, -1

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8


In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,28), forestList))
print(returnsList)

In [ ]:
indexes = []
for i in rebalanceIndexes(28,52):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(28,52), forestList))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
indexes = []
for i in rebalanceIndexes(52,76):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(52,76), forestList))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
indexes = []
for i in rebalanceIndexes(76,100):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(76,100), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [ ]:
indexes = []
for i in rebalanceIndexes(100,124):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(100,124), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [ ]:
indexes = []
for i in rebalanceIndexes(124,148):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(124,148), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [ ]:
indexes = []
for i in rebalanceIndexes(148,172):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(148,172), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [ ]:
indexes = []
for i in rebalanceIndexes(172,196):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(172,196), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [ ]:
print(returnsList)
print("Mean: " + str(np.mean(returnsList)))
print("Std Dev: " + str(np.std(returnsList)))
print("Total Sum: " + str(sum(returnsList)))
plt.hist(returnsList, 8, facecolor='green', alpha=0.75)